In [1]:
from argparse import Namespace
import xml.etree.ElementTree as ET
import os
from json import dumps
import xmltodict
from collections.abc import Iterable
from sqlalchemy import create_engine, inspect, Table, Column, ForeignKey, Integer, Boolean, Float, String, Text
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy.sql.expression import text
import pandas as pd
from json import dumps
import keyring
import google.generativeai as genai
import re
from contextlib import suppress

In [2]:
DB_FILEPATH  = './pdk_knowledge_base.db'
with suppress(FileNotFoundError):
    os.remove(DB_FILEPATH)

LYT_FILEPATH        = '$PDK_ROOT/$PDK/libs.tech/klayout/tech/$PDK.lyt'
DB_CONN_STR         = f'sqlite:///{DB_FILEPATH}'
INSERT_PDK_FILEPATH = './script.sql'

In [3]:
def xml_to_json(xml_filepath):
    xml_filepath = os.path.expandvars(xml_filepath)
    with open(xml_filepath) as f:
        s = f.read()
    return xmltodict.parse(s)

lyt = dumps(xml_to_json(LYT_FILEPATH), indent=4)
# print(lyt)

In [4]:
Base = declarative_base()

class Technology(Base):
    __tablename__     = 'technologies'
    id                = Column(Integer, primary_key=True)
    
    name              = Column(String(255), nullable=False, unique=True)
    description       = Column(Text, nullable=True)
    dbu               = Column(Float, nullable=False)
    add_other_layers  = Column(Boolean, nullable=False)
    
    writer_options    = relationship('WriterOptions', uselist=False, back_populates='technology')
    connections       = relationship('Connection', back_populates='technology')
    symbols           = relationship('Symbol', back_populates='technology')
    
class WriterOptions(Base):
    __tablename__    = 'writer_options'
    id               = Column(Integer, primary_key=True)
    
    libname          = Column(String(255), nullable=False)
    max_vertex_count = Column(Integer, nullable=False)
    write_timestamps = Column(Boolean, nullable=False)
    
    technology_id    = Column(Integer, ForeignKey('technologies.id'))
    technology       = relationship('Technology', back_populates='writer_options')

class Connection(Base):
    __tablename__ = 'connections'
    id            = Column(Integer, primary_key=True)
    
    a_symbol_id   = Column(Integer, ForeignKey('symbols.id'))
    a_symbol      = relationship('Symbol', foreign_keys=[a_symbol_id], back_populates='as_a_symbol_connections')
    
    via_symbol_id = Column(Integer, ForeignKey('symbols.id'))
    via_symbol    = relationship('Symbol', foreign_keys=[via_symbol_id], back_populates='as_via_symbol_connections')
    
    b_symbol_id   = Column(Integer, ForeignKey('symbols.id'))
    b_symbol      = relationship('Symbol', foreign_keys=[b_symbol_id], back_populates='as_b_symbol_connections')
    
    technology_id = Column(Integer, ForeignKey('technologies.id'))
    technology    = relationship('Technology', back_populates='connections')

SymbolPhysicalLayerPairs = Table(
    'symbol_physical_layer_pairs', 
    Base.metadata,
    Column('symbol_id', Integer, ForeignKey('symbols.id'), primary_key=True),
    Column('physical_layer_id', Integer, ForeignKey('physical_layers.id'), primary_key=True),
)

class Symbol(Base):
    __tablename__             = 'symbols'
    id                        = Column(Integer, primary_key=True)
    
    name                      = Column(String(255), nullable=False)
    
    technology_id             = Column(Integer, ForeignKey('technologies.id'))
    technology                = relationship('Technology', back_populates='symbols')
    
    as_a_symbol_connections   = relationship('Connection', foreign_keys=[Connection.a_symbol_id], back_populates='a_symbol')
    as_via_symbol_connections = relationship('Connection', foreign_keys=[Connection.via_symbol_id], back_populates='via_symbol')
    as_b_symbol_connections   = relationship('Connection', foreign_keys=[Connection.b_symbol_id], back_populates='b_symbol')
    
    physical_layers           = relationship('PhysicalLayer', secondary=SymbolPhysicalLayerPairs, back_populates='symbols')
    
class PhysicalLayer(Base):
    __tablename__ = 'physical_layers'
    id            = Column(Integer, primary_key=True)
    
    layer         = Column(Integer, nullable=False)
    datatype      = Column(Integer, nullable=False)
    negative      = Column(Boolean, server_default=text('0'), nullable=False)
    
    symbols       = relationship('Symbol', secondary=SymbolPhysicalLayerPairs, back_populates='physical_layers')

engine = create_engine(DB_CONN_STR, echo=False)
Base.metadata.create_all(engine)

In [5]:
def get_db_schema(db_conn_str):
    engine    = create_engine(db_conn_str)
    inspector = inspect(engine)
    schema    = []
    for table_name in inspector.get_table_names():
        s = f'TABLE: "{table_name}"\n\n'
        s += pd.DataFrame.from_records(
            inspector.get_columns(table_name)
        ).to_markdown(index=False)
        schema.append(s)
    schema = '\n\n---\n\n'.join(schema)
    return schema

db_schema = get_db_schema(DB_CONN_STR)
# print(db_schema)

In [6]:
def generate_pdk_insertion_script(db_schema, lyt, export_path):

    api_key = keyring.get_password('Gemini', 'default')
    genai.configure(api_key=api_key)
    
    model = genai.GenerativeModel('gemini-2.5-flash')
    response = model.generate_content(f"""
    TASK:
    ------
    Use the given SQLite SCHEMA and \
    JSON to generate SQLite queries \
    that insert the relevant data \
    from the JSON as records to \
    their respective tables from \
    the given SCHEMA.
    
    OUTPUT FORMAT:
    ---------------
    Do NOT explain anything, just \
    display the SQLite query \
    statements as a script, that's \
    it.
    
    ===============================
    
    SCHEMA:
    --------
    {db_schema}
    
    ===============================
    
    JSON:
    ------
    {lyt}
    """)

    sql_script = re.sub(r'```(sqlite)?', '', response.text).strip()
    with open(export_path, 'w') as f:
        f.write(sql_script)

    return sql_script

# _ = generate_pdk_insertion_script(db_schema, lyt, INSERT_PDK_FILEPATH)

In [7]:
def insert_pdk(db_conn_str, sql_script):
    try:
        if os.path.isfile(sql_script):
            with open(sql_script) as f:
                sql_script = f.read()
        engine = create_engine(db_conn_str)
        conn   = engine.raw_connection()
        cursor = conn.cursor()
        cursor.executescript(sql_script)
        conn.commit()
    finally:
        conn.close()

insert_pdk(DB_CONN_STR, INSERT_PDK_FILEPATH)